In [4]:
import cv2
import pytesseract
import numpy as np


image = cv2.imread('C:\\Users\\LENOVO\\Desktop\\materias\\POO\\video pytesseract\\codigo y imagenes poo\\ej1.jpg')




def reconocer_placa(image):
    pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract'
    imagen1 = cv2.resize(image, (800, 800))
    # Convertir la imagen a espacio de color HSV
    hsv = cv2.cvtColor(imagen1, cv2.COLOR_BGR2HSV)

    # Definir el rango de valores de matiz para el amarillo (ajusta estos valores según tus necesidades)
    lower_yellow = np.array([15, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Crear una máscara para aislar las regiones amarillas
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Encontrar contornos en la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtrar los contornos para eliminar los que no son relevantes
    filtered_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if 10000 < area < 500000:  # Ajusta los umbrales de área según tus necesidades
            filtered_contours.append(contour)

    # Dibujar los contornos filtrados en una copia de la imagen original
    image_with_contours = imagen1.copy()
    cv2.drawContours(image_with_contours, filtered_contours, -1, (0, 255, 0), 2)
    text=""
    # Realizar el recorte de las áreas amarillas
    yellow_regions = []
    for contour in filtered_contours:
        x, y, w, h = cv2.boundingRect(contour)
        yellow_region = imagen1[y:y+h, x:x+w]
        yellow_regions.append(yellow_region)

        # Aplicar suavizado
        yellow_blurred = cv2.GaussianBlur(yellow_region, (5, 5), 0)

        # Calcular la diferencia absoluta
        nB = np.matrix(yellow_blurred[:,:,0])
        nG = np.matrix(yellow_blurred[:,:,1])
        nR = np.matrix(yellow_blurred[:,:,2])
        Color = cv2.absdiff(nG, nB)

        # Umbralizar la imagen
        _, umbral = cv2.threshold(Color, 50, 255, cv2.THRESH_BINARY)

        # Aplicar eliminación de ruido (apertura)
        umbral = cv2.morphologyEx(umbral, cv2.MORPH_OPEN, kernel=np.ones((3, 3), np.uint8))

        # Filtrar contornos por tamaño
        contours, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        filtered_contours = [contour for contour in contours if 1000 < cv2.contourArea(contour) < 50000]

        # Dibujar contornos en una copia de la imagen original
        image_with_contours = yellow_region.copy()
        cv2.drawContours(image_with_contours, filtered_contours, -1, (0, 255, 0), 2)

        # Mostrar la imagen con contornos
        #cv2.imshow(f'PLACA ', umbral)
        #cv2.waitKey(0)
        
        text = pytesseract.image_to_string(umbral, config='--psm 11')
        print(f'PLACA : {text}')
        
       
        
    #cv2.imshow('Image', imagen1)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return text

placa = reconocer_placa(image)
print(placa)

PLACA : HJR 063

HJR 063

